This is the collection of codes that read food atlas datasets and CDC health indicator datasets from Github repository, integrate datasets and cleaning data

In [363]:
#merge food atlas datasets into one
import pandas as pd

Overall_folder='C:/Users/cathy/Capstone_project_1/'

dfs=list()
url_folder='https://raw.githubusercontent.com/cathyxinxyz/Capstone_Project_1/master/Datasets/Food_atlas/'

filenames=['ACCESS','ASSISTANCE','HEALTH','INSECURITY','LOCAL','PRICES_TAXES','RESTAURANTS','SOCIOECONOMIC','STORES']
for i,filename in enumerate(filenames):
    filepath=url_folder+filename+".csv"   
    d=pd.read_csv(filepath,index_col='FIPS',encoding="ISO-8859-1")
    #append datasets to the list and drop the redundent columns:'State' and 'County'
    if i!=0:
        dfs.append(d.drop(['State', 'County'], axis=1))
    else:
        dfs.append(d)

#merge datasets
df_merge=pd.concat(dfs, join='outer', axis=1)

In [364]:
print (df_merge.head(5))

     State   County  LACCESS_POP10  LACCESS_POP15  PCH_LACCESS_POP_10_15  \
FIPS                                                                       
1001    AL  Autauga   18428.439690   17496.693040              -5.056026   
1003    AL  Baldwin   35210.814080   30561.264430             -13.204891   
1005    AL  Barbour    5722.305602    6069.523628               6.067799   
1007    AL     Bibb    1044.867327     969.378841              -7.224696   
1009    AL   Blount    1548.175559    3724.428242             140.568857   

      PCT_LACCESS_POP10  PCT_LACCESS_POP15  LACCESS_LOWI10  LACCESS_LOWI15  \
FIPS                                                                         
1001          33.769657          32.062255     5344.427472     6543.676824   
1003          19.318473          16.767489     9952.144027     9886.831137   
1005          20.840972          22.105560     3135.676086     2948.790251   
1007           4.559753           4.230324      491.449066      596.162829   

Check columns for missing values

In [217]:
df_merge.describe()

,LACCESS_POP10,LACCESS_POP15,PCH_LACCESS_POP_10_15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,LACCESS_LOWI10,LACCESS_LOWI15,PCH_LACCESS_LOWI_10_15,PCT_LACCESS_LOWI10,PCT_LACCESS_LOWI15,...,PCH_SNAPS_12_16,SNAPSPTH12,SNAPSPTH16,PCH_SNAPSPTH_12_16,WICS08,WICS12,PCH_WICS_08_12,WICSPTH08,WICSPTH12,PCH_WICSPTH_08_12
count,3143.000000,3124.000000,3.117000e+03,3143.000000,3124.000000,3143.000000,3123.000000,3.115000e+03,3143.000000,3123.000000,...,3140.000000,3143.000000,3141.000000,3138.000000,3143.000000,3143.000000,3124.000000,3143.000000,3143.000000,3124.000000
mean,20156.653242,19287.747386,7.180008e+03,23.559330,23.084470,5551.776559,5833.959524,7.318957e+03,8.374953,8.651256,...,19.774565,0.880064,1.025253,20.079051,14.371619,15.109131,-1.901222,0.255593,0.230473,-3.995745
std,51308.865791,47803.525595,3.955676e+05,20.250176,19.602745,13862.922934,13989.348252,4.004692e+05,8.214988,8.368703,...,32.074488,0.387638,0.451670,39.262939,49.856567,55.831489,25.937208,0.257250,0.211762,25.580415
min,0.000000,0.000000,-1.000000e+02,0.000000,0.000000,0.000000,0.000000,-1.000000e+02,0.000000,0.000000,...,-100.000000,0.000000,0.000000,-100.000000,0.000000,0.000000,-100.000000,0.000000,0.000000,-100.000000
25%,1661.076793,1631.358727,-1.091343e+01,10.847099,10.927326,592.983070,608.235919,-1.232638e+01,3.442171,3.625438,...,5.405406,0.629603,0.750265,5.136099,3.000000,3.000000,-12.500000,0.123272,0.118920,-15.333640
50%,4097.827186,4131.174278,-6.239322e-02,19.685956,19.200682,1500.780036,1636.858816,3.486028e+00,6.150772,6.508531,...,15.915190,0.818400,0.965336,15.206350,5.000000,5.000000,0.000000,0.189785,0.177336,-2.383872
75%,12954.123205,13119.029393,7.262080e+00,29.581054,28.862231,4591.067541,4859.016156,2.352342e+01,10.324935,10.679796,...,28.979172,1.066786,1.234080,28.328855,11.000000,11.000000,0.000000,0.296110,0.268129,3.158470
max,886068.668400,740267.412700,2.208376e+07,100.000001,100.000001,292541.789000,259479.516000,2.234893e+07,72.274456,71.844209,...,1100.000000,6.658001,6.695621,1135.312012,1285.000000,1602.000000,300.000000,4.618937,2.994012,294.070300


In [218]:
number_null_values_percol=df_merge.isnull().sum(axis=0)
#columns with over 100 missing values
cols_with_over_10_percent_null_values=number_null_values_percol[number_null_values_percol>0.1*df_merge.shape[0]]
print (cols_with_over_10_percent_null_values.index)

Index(['PC_WIC_REDEMP08', 'PC_WIC_REDEMP12', 'PCH_PC_WIC_REDEMP_08_12',
       'REDEMP_WICS08', 'REDEMP_WICS12', 'PCH_REDEMP_WICS_08_12', 'PCT_HSPA15',
       'PCT_LOCLSALE07', 'PCH_DIRSALES_07_12', 'PCH_PC_DIRSALES_07_12',
       'PCH_FMRKT_09_16', 'PCH_FMRKTPTH_09_16', 'FMRKT_SNAP16',
       'PCT_FMRKT_SNAP16', 'FMRKT_WIC16', 'PCT_FMRKT_WIC16', 'FMRKT_WICCASH16',
       'PCT_FMRKT_WICCASH16', 'FMRKT_SFMNP16', 'PCT_FMRKT_SFMNP16',
       'FMRKT_CREDIT16', 'PCT_FMRKT_CREDIT16', 'FMRKT_FRVEG16',
       'PCT_FMRKT_FRVEG16', 'FMRKT_ANMLPROD16', 'PCT_FMRKT_ANMLPROD16',
       'FMRKT_BAKED16', 'PCT_FMRKT_BAKED16', 'FMRKT_OTHERFOOD16',
       'PCT_FMRKT_OTHERFOOD16', 'PCH_VEG_FARMS_07_12', 'VEG_ACRES07',
       'VEG_ACRES12', 'PCH_VEG_ACRES_07_12', 'VEG_ACRESPTH07',
       'VEG_ACRESPTH12', 'PCH_VEG_ACRESPTH_07_12', 'PCH_FRESHVEG_FARMS_07_12',
       'FRESHVEG_ACRES07', 'FRESHVEG_ACRES12', 'PCH_FRESHVEG_ACRES_07_12',
       'FRESHVEG_ACRESPTH07', 'FRESHVEG_ACRESPTH12',
       'PCH_FRESHVEG_A

In [219]:
#drop these columns first
df_merge=df_merge.drop(list(cols_with_over_10_percent_null_values.index), axis=1)

In [220]:
df_merge.shape

(3143, 209)

In [221]:
#check number of remaining columns
print (df_merge.columns)

Index(['State', 'County', 'LACCESS_POP10', 'LACCESS_POP15',
       'PCH_LACCESS_POP_10_15', 'PCT_LACCESS_POP10', 'PCT_LACCESS_POP15',
       'LACCESS_LOWI10', 'LACCESS_LOWI15', 'PCH_LACCESS_LOWI_10_15',
       ...
       'PCH_SNAPS_12_16', 'SNAPSPTH12', 'SNAPSPTH16', 'PCH_SNAPSPTH_12_16',
       'WICS08', 'WICS12', 'PCH_WICS_08_12', 'WICSPTH08', 'WICSPTH12',
       'PCH_WICSPTH_08_12'],
      dtype='object', length=209)


categorizes columns into three groups: category data ('State' and 'County'), count data, percent data, # per 1000 pop, and percent change

columns to keep: category data ('State' and 'County'), percent data, # per 1000 pop, and percent change; remove count data because it is not adjusted by population size

Each column name is highly abstract and unreadable, need to extract info from the variable information provided by Food_atlas

In [222]:
url='https://raw.githubusercontent.com/cathyxinxyz/Capstone_Project_1/master/Datasets/Food_atlas/variable_info.csv'
var_info_df=pd.read_csv(url,encoding="ISO-8859-1", index_col='Variable Code')

further filter varaibles based on following principles:
i. keep variables that are adjusted by population size: '% change', 'Percent', '# per 1,000 pop','Percentage points';
ii. keep variables that are mostly valuable for analysis
iii. keep variables where values are valid: e.g. no negative values for variables with units as 'Percent' or '# per 1,000 pop'.


In [223]:
#units to keep: '% change', 'Percent', '# per 1,000 pop','Percentage points'
#var_info_df['Units'].isin(['Percent', '# per 1,000 pop','Dollars'])
var_info_df_subset=var_info_df[var_info_df['Units'].isin(['Percent', '# per 1,000 pop','Dollars'])]
var_subset=list(var_info_df_subset.index)
var_subset.extend(['State', 'County'])
#print (var_subset)
df_subset=df_merge.loc[:, var_subset]
#print (df_merge.shape)

In [224]:
print (df_subset.shape)

(3143, 106)


In [225]:
#check weather each column has valid values:
####### columns with units 'Percent' should have values between 0 and 100, any value that fall out of this range should be changed to NaN values
###### 
######
######

#Replace invalid values with np.nan
import numpy as np

for c in df_subset.columns: 
    if c in var_info_df.index:
        if var_info_df.loc[c]['Units'] =='Percent': 
            df_subset[c][(df_subset[c]<0)|(df_subset[c]>100)]=np.nan
        elif var_info_df.loc[c]['Units'] =='# per 1,000 pop':
            df_subset[c][(df_subset[c]<0)|(df_subset[c]>1000)]=np.nan
        elif var_info_df.loc[c]['Units'] =='Dollars':
            df_subset[c][(df_subset[c]<0)]=np.nan


        

C:\anaconda\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\anaconda\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\anaconda\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [226]:
#break df_tp into two sets: variables measured at the earlier time point: df_tp_1; and variables measured at the earlier time point: df_tp_2

#group the same measure into tuples, the same measure share the same name except the last two digits which indicate the year of the measure
var_grouped_by_measures=defaultdict(list)
measures=list()
for var in df_subset.columns:
    if var not in ['State','County']:
        var_grouped_by_measures[(var_info_df.loc[var]['Category Name'],var_info_df.loc[var]['Sub_subcategory Name'])].append(var)
#df_subset=df_subset[measures]

In [227]:
n=1  
var_info=list()
vars_to_keep=list()
for measures in var_grouped_by_measures.values():
    var_name='var'+str(n)
    df_subset[var_name]=sum([df_subset[m] for m in measures])/len(measures)
    var_info.append([var_name, var_info_df.loc[measures[0]]['Category Name'],
                     var_info_df.loc[measures[0]]['Category Code'],
                     var_info_df.loc[measures[0]]['Subcategory Name'],
                     var_info_df.loc[measures[0]]['Sub_subcategory Name'],
                     var_info_df.loc[measures[0]]['Units']])
    vars_to_keep.append('var'+str(n))
    n+=1

In [228]:
df_subset[['PCT_LACCESS_POP10','PCT_LACCESS_POP15','var1']].head(5)

,PCT_LACCESS_POP10,PCT_LACCESS_POP15,var1
FIPS,,,
1001,33.769657,32.062255,32.915956
1003,19.318473,16.767489,18.042981
1005,20.840972,22.105560,21.473266
1007,4.559753,4.230324,4.395039
1009,2.700840,6.497380,4.599110


In [229]:
var_name_info_df=pd.DataFrame(np.array(var_info))
var_name_info_df.columns=['var_name', 'Category Name','Category Code', 'Subcategory Name', 'Sub_subcategory Name','Units']
var_name_info_df.set_index('var_name')


,Category Name,Category Code,Subcategory Name,Sub_subcategory Name,Units
var_name,,,,,
var1,Access and Proximity to Grocery Store,ACCESS,Overall,Overall,Percent
var2,Access and Proximity to Grocery Store,ACCESS,Household Resources,Low income,Percent
var3,Access and Proximity to Grocery Store,ACCESS,Household Resources,no car,Percent
var4,Access and Proximity to Grocery Store,ACCESS,Household Resources,SNAP,Percent
var5,Access and Proximity to Grocery Store,ACCESS,Age_group,Children,Percent
var6,Access and Proximity to Grocery Store,ACCESS,Age_group,Seniors,Percent
var7,Access and Proximity to Grocery Store,ACCESS,Race,White,Percent
var8,Access and Proximity to Grocery Store,ACCESS,Race,Black,Percent
var9,Access and Proximity to Grocery Store,ACCESS,Race,Hispanic ethnicity,Percent


In [230]:
var_name_info_df.to_csv('C:/Users/cathy/Capstone_project_1/Datasets/Food_atlas/Var_name_info.csv')

In [231]:
#further truncate less relevant variables: var25,26,42,43,45-57
var_id_to_prune=[25,26,42,43]
var_id_to_prune.extend(range(45,58))
vars_subset=list(set(vars_to_keep).difference(['var'+str(i) for i in var_id_to_prune]))

In [232]:
df_subset=df_subset[vars_subset]
df_subset.shape

(3143, 52)

In [233]:
df_subset.to_csv(Overall_folder+'Datasets/food_environment.csv')

Integrate CDC Datasets together

In [234]:
import pandas as pd
dfs=list()
sub_folder=Overall_folder+'/Datasets/CDC/'
filenames=['Diabetes_prevalence',
           'Obesity_prevalence',
           'Physical_inactive_prevalence']

In [235]:
for filename in filenames:
    filepath=sub_folder+filename+".csv"   
    df=pd.read_csv(filepath,index_col='FIPS')
    
    
    if 'Diabetes' in filename:
        df.columns=df.columns.astype(str)+'_db'
    elif 'Obesity' in filename:
        df.columns=df.columns.astype(str)+'_ob'
    elif 'Physical' in filename:
        df.columns=df.columns.astype(str)+'_phy'
    dfs.append(df)
#merge datasets


In [236]:
CDC_merge=pd.concat(dfs, join='outer', axis=1)

In [237]:
CDC_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3224 entries, 1001 to 72153
Data columns (total 30 columns):
2004_db     3224 non-null object
2005_db     3224 non-null object
2006_db     3224 non-null object
2007_db     3224 non-null object
2008_db     3224 non-null object
2009_db     3224 non-null object
2010_db     3224 non-null object
2011_db     3224 non-null object
2012_db     3224 non-null object
2013_db     3224 non-null object
2004_ob     3224 non-null object
2005_ob     3224 non-null object
2006_ob     3224 non-null object
2007_ob     3224 non-null object
2008_ob     3224 non-null object
2009_ob     3224 non-null object
2010_ob     3224 non-null object
2011_ob     3146 non-null object
2012_ob     3146 non-null object
2013_ob     3146 non-null object
2004_phy    3224 non-null object
2005_phy    3224 non-null object
2006_phy    3224 non-null object
2007_phy    3224 non-null object
2008_phy    3224 non-null object
2009_phy    3224 non-null object
2010_phy    3224 non-null objec

In [238]:
#Find out the non numeric entries in CDC_merge
for c in CDC_merge.columns:
    num_non_numeric=sum(CDC_merge.applymap(lambda x: isinstance(x, (int, float)))[c])
    if num_non_numeric>0:
        print(c, num_non_numeric, CDC_merge[pd.to_numeric(CDC_merge[c], errors='coerce').isnull()])

2011_ob 78       2004_db             2005_db 2006_db 2007_db 2008_db  2009_db  2010_db  \
FIPS                                                                          
2201      5.8  6.1000000000000005     5.9     6.8     7.4  No Data  No Data   
2232      5.8  6.1000000000000005     6.6     6.9     5.2  No Data  No Data   
2280      5.4                 6.2     6.4       7       7  No Data  No Data   
72001    11.4                10.9    12.8    12.7    14.3     14.7     15.4   
72003    12.4                14.9    16.3    15.5    14.7     14.6     16.4   
72005    12.8                14.6    16.3    14.9    15.4       15     15.9   
72007    11.1                12.9      14    14.7    14.7     15.1       15   
72009      13                13.6    13.7    13.2    13.2     12.5     13.9   
72011    12.1                13.8    15.3    14.9    14.7     14.8     16.9   
72013    13.8                14.5    14.8    15.3    16.4     17.9     17.2   
72015    13.6                13.9    13.4

2013_ob 78                  2004_db             2005_db 2006_db 2007_db 2008_db  2009_db  \
FIPS                                                                            
2201                 5.8  6.1000000000000005     5.9     6.8     7.4  No Data   
2232                 5.8  6.1000000000000005     6.6     6.9     5.2  No Data   
2280                 5.4                 6.2     6.4       7       7  No Data   
51515  9.700000000000001  10.200000000000001    11.1    12.1    12.6     12.7   
72001               11.4                10.9    12.8    12.7    14.3     14.7   
72003               12.4                14.9    16.3    15.5    14.7     14.6   
72005               12.8                14.6    16.3    14.9    15.4       15   
72007               11.1                12.9      14    14.7    14.7     15.1   
72009                 13                13.6    13.7    13.2    13.2     12.5   
72011               12.1                13.8    15.3    14.9    14.7     14.8   
72013            

2012_phy 78       2004_db             2005_db 2006_db 2007_db 2008_db  2009_db  2010_db  \
FIPS                                                                          
2201      5.8  6.1000000000000005     5.9     6.8     7.4  No Data  No Data   
2232      5.8  6.1000000000000005     6.6     6.9     5.2  No Data  No Data   
2280      5.4                 6.2     6.4       7       7  No Data  No Data   
72001    11.4                10.9    12.8    12.7    14.3     14.7     15.4   
72003    12.4                14.9    16.3    15.5    14.7     14.6     16.4   
72005    12.8                14.6    16.3    14.9    15.4       15     15.9   
72007    11.1                12.9      14    14.7    14.7     15.1       15   
72009      13                13.6    13.7    13.2    13.2     12.5     13.9   
72011    12.1                13.8    15.3    14.9    14.7     14.8     16.9   
72013    13.8                14.5    14.8    15.3    16.4     17.9     17.2   
72015    13.6                13.9    13.

2013_phy 78                  2004_db             2005_db 2006_db 2007_db 2008_db  2009_db  \
FIPS                                                                            
2201                 5.8  6.1000000000000005     5.9     6.8     7.4  No Data   
2232                 5.8  6.1000000000000005     6.6     6.9     5.2  No Data   
2280                 5.4                 6.2     6.4       7       7  No Data   
51515  9.700000000000001  10.200000000000001    11.1    12.1    12.6     12.7   
72001               11.4                10.9    12.8    12.7    14.3     14.7   
72003               12.4                14.9    16.3    15.5    14.7     14.6   
72005               12.8                14.6    16.3    14.9    15.4       15   
72007               11.1                12.9      14    14.7    14.7     15.1   
72009                 13                13.6    13.7    13.2    13.2     12.5   
72011               12.1                13.8    15.3    14.9    14.7     14.8   
72013           

In [239]:
#It turns out that some entries are 'No Data' or NaN, so I replace the 'No Data' with NaN values
CDC_merge=CDC_merge.replace('No Data', np.nan)
CDC_merge=CDC_merge.astype(float)

In [240]:
#now check the CDC_merge
CDC_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3224 entries, 1001 to 72153
Data columns (total 30 columns):
2004_db     3219 non-null float64
2005_db     3219 non-null float64
2006_db     3219 non-null float64
2007_db     3219 non-null float64
2008_db     3219 non-null float64
2009_db     3221 non-null float64
2010_db     3221 non-null float64
2011_db     3221 non-null float64
2012_db     3221 non-null float64
2013_db     3220 non-null float64
2004_ob     3141 non-null float64
2005_ob     3141 non-null float64
2006_ob     3141 non-null float64
2007_ob     3141 non-null float64
2008_ob     3141 non-null float64
2009_ob     3143 non-null float64
2010_ob     3143 non-null float64
2011_ob     3143 non-null float64
2012_ob     3143 non-null float64
2013_ob     3142 non-null float64
2004_phy    3141 non-null float64
2005_phy    3141 non-null float64
2006_phy    3141 non-null float64
2007_phy    3141 non-null float64
2008_phy    3141 non-null float64
2009_phy    3143 non-null float64
2010_

In [250]:
#choose the latest prevalence of diabetes, obesity and physical inactivity to merge with df_tp 
CDC_subset=CDC_merge[['2013_db','2013_ob','2011_phy','2012_phy','2013_phy']]

In [251]:
CDC_subset['prevalence of physical inactivity']=(CDC_subset['2011_phy']+CDC_subset['2012_phy']+CDC_subset['2013_phy'])/3

C:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [252]:
CDC_subset.head(5)

,2013_db,2013_ob,2011_phy,2012_phy,2013_phy,prevalence of physical inactivity
FIPS,,,,,,
1001,13.0,34.1,27.9,28.7,28.6,28.400000
1003,10.4,27.4,25.2,25.2,22.3,24.233333
1005,18.4,44.4,32.3,32.9,31.8,32.333333
1007,14.8,40.3,33.2,33.2,33.9,33.433333
1009,14.1,34.6,29.8,28.5,28.0,28.766667


In [253]:
CDC_subset.rename(columns={'2013_db': 'prevalence of diabetes', '2013_ob': 'prevalence of obesity'}, inplace=True)

C:\anaconda\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [254]:
CDC_subset[['prevalence of diabetes', 'prevalence of obesity', 'prevalence of physical inactivity']].to_csv(Overall_folder+'Datasets/Db_ob_phy.csv')

Integrating geography dataset

In [337]:
df=pd.read_excel(Overall_folder+'Datasets/geography/ruralurbancodes2013.xls')

In [338]:
df.head(5)

,FIPS,State,County_Name,Population_2010,RUCC_2013,Description
0,1001,AL,Autauga County,54571,2,"Metro - Counties in metro areas of 250,000 to ..."
1,1003,AL,Baldwin County,182265,3,Metro - Counties in metro areas of fewer than ...
2,1005,AL,Barbour County,27457,6,"Nonmetro - Urban population of 2,500 to 19,999..."
3,1007,AL,Bibb County,22915,1,Metro - Counties in metro areas of 1 million p...
4,1009,AL,Blount County,57322,1,Metro - Counties in metro areas of 1 million p...


In [339]:
df=df.set_index('FIPS')

In [340]:
df.to_csv(Overall_folder+'Datasets/rural_urban_codes.csv')
df[['RUCC_2013']].to_csv(Overall_folder+'Datasets/RUCC_codes.csv')

clean risk factors datasets

In [341]:
filenames=['DEMOGRAPHICS','RISKFACTORSANDACCESSTOCARE']

dfs=list()
for i,filename in enumerate(filenames):
    filepath=Overall_folder+'Datasets/CHSI_dataset/'+filename+".csv"   
    d=pd.read_csv(filepath)
    #append datasets to the list and drop the redundent columns:'State' and 'County'
    
    dfs.append(d)

In [342]:
print (dfs[0].head(10))

   State_FIPS_Code  County_FIPS_Code CHSI_County_Name CHSI_State_Name  \
0                1                 1          Autauga         Alabama   
1                1                 3          Baldwin         Alabama   
2                1                 5          Barbour         Alabama   
3                1                 7             Bibb         Alabama   
4                1                 9           Blount         Alabama   
5                1                11          Bullock         Alabama   
6                1                13           Butler         Alabama   
7                1                15          Calhoun         Alabama   
8                1                17         Chambers         Alabama   
9                1                19         Cherokee         Alabama   

  CHSI_State_Abbr  Strata_ID_Number  \
0              AL                29   
1              AL                16   
2              AL                51   
3              AL                42   
4

In [343]:
print (dfs[1].head(10))

   State_FIPS_Code  County_FIPS_Code CHSI_County_Name CHSI_State_Name  \
0                1                 1          Autauga         Alabama   
1                1                 3          Baldwin         Alabama   
2                1                 5          Barbour         Alabama   
3                1                 7             Bibb         Alabama   
4                1                 9           Blount         Alabama   
5                1                11          Bullock         Alabama   
6                1                13           Butler         Alabama   
7                1                15          Calhoun         Alabama   
8                1                17         Chambers         Alabama   
9                1                19         Cherokee         Alabama   

  CHSI_State_Abbr  Strata_ID_Number  No_Exercise  CI_Min_No_Exercise  \
0              AL                29         27.8                20.7   
1              AL                16         27.2    

In [344]:
dfs[1][dfs[1]<0]=np.nan

In [345]:
dfs[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3141 entries, 0 to 3140
Data columns (total 31 columns):
State_FIPS_Code                3141 non-null int64
County_FIPS_Code               3141 non-null int64
CHSI_County_Name               0 non-null object
CHSI_State_Name                0 non-null object
CHSI_State_Abbr                0 non-null object
Strata_ID_Number               3141 non-null int64
No_Exercise                    2206 non-null float64
CI_Min_No_Exercise             2206 non-null float64
CI_Max_No_Exercise             2206 non-null float64
Few_Fruit_Veg                  1904 non-null float64
CI_Min_Fruit_Veg               1904 non-null float64
CI_Max_Fruit_Veg               1904 non-null float64
Obesity                        2224 non-null float64
CI_Min_Obesity                 2224 non-null float64
CI_Max_Obesity                 2224 non-null float64
High_Blood_Pres                1522 non-null float64
CI_Min_High_Blood_Pres         1522 non-null float64
CI_Max_Hig

In [346]:
#combine State FIPS code and county FIPS code to be FIPS code
#state FIPS code is interpreted as integer values so the zero before some single digits are ignored
# similarly, county FIPS code is also interpreted as integers
# to combine both as FIPS code, we first need to tranform both codes to string and fill in the zeros
# to make all state FIPS code to be two digits and all county FIPS code to be three digits
dfs[0]['FIPS']=dfs[0]['State_FIPS_Code'].apply((lambda x:('0'+str(x))[-2:]))+dfs[0]['County_FIPS_Code'].apply((lambda x:('00'+str(x))[-3:]))
dfs[1]['FIPS']=dfs[1]['State_FIPS_Code'].apply((lambda x:('0'+str(x))[-2:]))+dfs[1]['County_FIPS_Code'].apply((lambda x:('00'+str(x))[-3:]))

In [347]:
#I need the population size info from demographic file
dfs[0]["Population_Size"].describe()

count    3.141000e+03
mean     9.436816e+04
std      3.064317e+05
min      6.200000e+01
25%      1.121100e+04
50%      2.523500e+04
75%      6.404000e+04
max      9.935475e+06
Name: Population_Size, dtype: float64

In [348]:
#I need the number of uninsured population info from risk factors file
dfs[1]['Uninsured'].describe()

count    3.138000e+03
mean     1.265902e+04
std      5.462895e+04
min      1.100000e+01
25%      1.551000e+03
50%      3.430000e+03
75%      8.117500e+03
max      2.167891e+06
Name: Uninsured, dtype: float64

In [349]:
sum(dfs[1]['Uninsured'].isnull())
#there are three missing values, which is pretty small number as compared to over 3000 counties


3

In [350]:
dfs[0]=dfs[0].set_index('FIPS')
dfs[1]=dfs[1].set_index('FIPS')

In [351]:
df_combined=pd.concat([dfs[0][['Population_Size']], dfs[1][['Uninsured']]], axis=1, join='inner')

In [352]:
df_combined

,Population_Size,Uninsured
FIPS,,
01001,48612,5690.0
01003,162586,19798.0
01005,28414,5126.0
01007,21516,3315.0
01009,55725,8131.0
01011,11055,2295.0
01013,20766,4055.0
01015,112141,16456.0
01017,35460,5533.0


In [353]:
df_combined['frac_uninsured']=df_combined['Uninsured']/df_combined['Population_Size']

In [354]:
df_combined

,Population_Size,Uninsured,frac_uninsured
FIPS,,,
01001,48612,5690.0,0.117049
01003,162586,19798.0,0.121769
01005,28414,5126.0,0.180404
01007,21516,3315.0,0.154071
01009,55725,8131.0,0.145913
01011,11055,2295.0,0.207598
01013,20766,4055.0,0.195271
01015,112141,16456.0,0.146744
01017,35460,5533.0,0.156035


In [355]:
df_combined[['frac_uninsured']].to_csv(Overall_folder+'Datasets/Uninsured.csv')

Integrate all datasets

In [357]:
filenames=['food_environment', 'Db_ob_phy', 'Uninsured', 'RUCC_codes']
dfs=list()
for filename in filenames:
    df=pd.read_csv(Overall_folder+'Datasets/'+filename+'.csv', index_col='FIPS')
    dfs.append(df)
    
df_merge=pd.concat(dfs, axis=1, join='inner')

In [358]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3138 entries, 1001 to 56045
Data columns (total 57 columns):
var68                                3134 non-null float64
var44                                3137 non-null float64
var60                                3138 non-null float64
var14                                3138 non-null float64
var13                                3119 non-null float64
var39                                3138 non-null float64
var9                                 3119 non-null float64
var67                                3134 non-null float64
var3                                 3135 non-null float64
var40                                3138 non-null float64
var23                                3138 non-null float64
var8                                 3119 non-null float64
var18                                3136 non-null float64
var2                                 3118 non-null float64
var65                                3138 non-null float64
var2

In [359]:
df_merge.to_csv(Overall_folder+'Datasets/combined.csv')

combine state, county, fips code file into one for map

In [370]:
df=pd.read_csv(Overall_folder+'Datasets/Food_atlas/Supplemental_data_county.csv',encoding="ISO-8859-1", index_col='FIPS')

In [369]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 10 columns):
FIPS                         3142 non-null int64
State                        3142 non-null object
County                       3142 non-null object
2010 Census Population       3142 non-null object
Population Estimate, 2011    3142 non-null object
Population Estimate, 2012    3142 non-null object
Population Estimate, 2013    3142 non-null object
Population Estimate, 2014    3142 non-null object
Population Estimate, 2015    3142 non-null object
Population Estimate, 2016    3142 non-null object
dtypes: int64(1), object(9)
memory usage: 245.5+ KB


In [374]:
df['State']=df['State'].apply((lambda x:x.lower()))
df['County']=df['County'].apply((lambda x:x.lower()))

In [376]:
df[['State', 'County']].to_csv(Overall_folder+'Datasets/state_county_name.csv')